In [3]:
import argparse
import os
import random
import sys
import numpy as np
import torch
import torchaudio.functional as F
import config

from utils import download_dataset_if_needed

sys.path.insert(0, "models/VGGish")

from models.VGGish.vggish_multichannel import VGGishMulti

import pandas as pd
import numpy as np

import os

sys.path.insert(0, 'datasets/')

from datasets.dataset import Dataset
from datasets import livingroom
from datasets.roomsetup import RoomSetup
import matplotlib.pyplot as plt

import librosa

from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split

from utils import download_dataset_if_needed

In [4]:
download_dataset_if_needed()

gvimont/diffusion/hackathon-minarm-2024/Acoustique/LivingRoom_preprocessed_hack.zip


In [6]:
DATASET_PATH = "LivingRoom_preprocessed_hack/Human1"

dr = Dataset(RoomSetup(livingroom.speaker_xyz,
                livingroom.mic_xyzs,
                livingroom.x_min,
                livingroom.x_max,
                livingroom.y_min,
                livingroom.y_max,
                livingroom.walls), DATASET_PATH)
livingroom.mic_xyzs

array([[-3343.275 , -3146.425 ,  1277.9375],
       [-3284.5375,  1587.5   ,  1277.9375],
       [  136.525 ,    53.975 ,  1277.9375],
       [  -76.2   , -3594.1   ,  1277.9375]])

In [7]:
centroid = np.load(os.path.join(DATASET_PATH, "centroid.npy"))
print("Shape of Centroid:")
print(centroid.shape)

#Loading Room Impulse Response (1000 human locations x 10 microphones x M time samples)
deconv = np.load(os.path.join(DATASET_PATH, "deconvoled_trim.npy"), mmap_mode='r')
print("Shape of RIRs:")
print(deconv.shape)

Shape of Centroid:
(1000, 2)
Shape of RIRs:
(1000, 4, 667200)


In [8]:
save_path = "models/VGGish/vggish_multi_model.pt"
error_path ="models/VGGish/vggish_multi_errors/"

vggish_multi = VGGishMulti(save_path=save_path, error_path=error_path)
# vggish_multi = VGGishMulti(saved_model="models/VGGish/vggish_multi_model.pt", save_path=save_path, error_path=error_path)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(deconv, centroid, test_size=0.2, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [10]:
# Train the VGGishMulti model
vggish_multi.fit(X_train, y_train, X_valid, y_valid, num_epochs=2)

Reshuffling for Epoch 0


/opt/mamba/lib/python3.11/site-packages/torch/functional.py:660: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:874.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


Validation XY Loss: 0.983
Validation Loss: 0.983
Reshuffling for Epoch 1
Validation XY Loss: 0.982
Validation Loss: 0.982


In [11]:
vggish_multi.predict(X_test, y_test)

TEST ERROR
[1273.3767   159.25922 1772.5337  1510.1097  1423.9376  2471.7148
 1942.2927  2281.0784  2926.3496  2295.0928  2372.766    986.07697
 2536.6697  2404.627   2267.8706  1644.9401  1164.815   1643.2407
 1152.3745  2070.6655  2428.667   1920.084   1180.7115  2060.5713
 1046.756   1847.9352  1818.9242  1710.5706  2209.      1773.8237
  947.43866 2326.92    2152.7578  2339.6025  1312.389   1949.9246
 1937.6398  1742.6042  2200.3057   717.5978   899.3248  2282.3188
 2354.338   2170.457   2454.4822  1146.793   1544.7355  2165.8862
 2152.913   1595.2144   960.19836 1397.1478   317.97366 1169.3341
 2273.7693   935.0873  1465.459   1697.2299  1178.6788  2085.3584
 2194.8162   525.28564 2730.5215   782.21606 2350.748    359.75012
 1763.6068  1754.4211  2190.853   2397.7383  1675.1681  1890.7876
  824.5244  2298.8464  1577.5557  2261.1587  1832.2878  1353.4713
 1105.8882  1921.6786  1374.7019  1995.9894  1550.3176  2110.8901
 1957.0685  1699.4635  1767.4186  1417.5797  1425.505    806.47